# Discovering Ramanujan's Partition Congruences

The **partition function** $p(n)$ counts the number of ways to write $n$ as a sum of positive
integers (order does not matter). For example, $p(4) = 5$ because:

$$4 = 3+1 = 2+2 = 2+1+1 = 1+1+1+1$$

Ramanujan discovered three remarkable congruences:

- $p(5n+4) \equiv 0 \pmod{5}$
- $p(7n+5) \equiv 0 \pmod{7}$
- $p(11n+6) \equiv 0 \pmod{11}$

In this notebook, we use **q-Kangaroo** to compute partition values, generate
partition generating functions, and automatically discover these congruences.

In [1]:
from q_kangaroo import QSession, partition_count

# Compute partition values p(0) through p(15)
for n in range(16):
    print(f"p({n}) = {partition_count(n)}")

p(0) = 1
p(1) = 1
p(2) = 2
p(3) = 3
p(4) = 5
p(5) = 7
p(6) = 11
p(7) = 15
p(8) = 22
p(9) = 30
p(10) = 42
p(11) = 56
p(12) = 77
p(13) = 101
p(14) = 135
p(15) = 176


## Partition Generating Function

The partition generating function is the formal power series

$$\sum_{n=0}^{\infty} p(n)\, q^n = \prod_{k=1}^{\infty} \frac{1}{1 - q^k}$$

We can compute this directly as a truncated series:

In [2]:
from q_kangaroo import partition_gf

s = QSession()
pgf = partition_gf(s, 25)
pgf

1 + q + 2*q^2 + 3*q^3 + 5*q^4 + 7*q^5 + 11*q^6 + 15*q^7 + 22*q^8 + 30*q^9 + 42*q^10 + 56*q^11 + 77*q^12 + 101*q^13 + 135*q^14 + 176*q^15 + 231*q^16 + 297*q^17 + 385*q^18 + 490*q^19 + 627*q^20 + 792*q^21 + 1002*q^22 + 1255*q^23 + 1575*q^24 + O(q^25)

## Extracting Subsequences with `sift`

Ramanujan's first congruence says $p(5n+4) \equiv 0 \pmod{5}$. We can extract the
subsequence $p(5n+4)$ using `sift(f, m, j)`, which picks out the coefficient of
$q^{mn+j}$ from a series $f$:

In [3]:
from q_kangaroo import sift

# Compute partition_gf to high order, then extract p(5n+4)
pgf50 = partition_gf(s, 50)
sifted = sift(pgf50, 5, 4)
sifted

5 + 30*q + 135*q^2 + 490*q^3 + 1575*q^4 + 4565*q^5 + 12310*q^6 + 31185*q^7 + 75175*q^8 + 173525*q^9 + O(q^10)

Every coefficient (5, 30, 135, 490, 1575, ...) is divisible by 5!

## Automatic Congruence Discovery

The `findcong` function systematically searches for congruences of the form
$p(mn+r) \equiv 0 \pmod{d}$. Let's search with moduli 2, 3, 5, 7, 11:

In [4]:
from q_kangaroo import findcong

# Search for congruences with moduli [2, 3, 5, 7, 11]
pgf100 = partition_gf(s, 100)
congruences = findcong(pgf100, [2, 3, 5, 7, 11])

for c in congruences:
    print(f"p({c['modulus']}n + {c['residue']}) == 0 (mod {c['divisor']})")

p(5n + 4) == 0 (mod 5)
p(7n + 5) == 0 (mod 7)
p(11n + 6) == 0 (mod 11)


q-Kangaroo automatically rediscovers all three of Ramanujan's classical congruences.

## Verification

Let's verify the first congruence directly:

In [5]:
# Verify p(5n+4) mod 5 = 0 for n = 0, 1, ..., 9
for n in range(10):
    val = partition_count(5 * n + 4)
    print(f"p({5*n+4}) = {val}, mod 5 = {val % 5}")

p(4) = 5, mod 5 = 0
p(9) = 30, mod 5 = 0
p(14) = 135, mod 5 = 0
p(19) = 490, mod 5 = 0
p(24) = 1575, mod 5 = 0
p(29) = 4565, mod 5 = 0
p(34) = 12310, mod 5 = 0
p(39) = 31185, mod 5 = 0
p(44) = 75175, mod 5 = 0
p(49) = 173525, mod 5 = 0


## Rank Generating Function

Dyson's **rank** of a partition is defined as the largest part minus the number of parts.
The rank generating function is:

$$R(z, q) = 1 + \sum_{n=1}^{\infty} \frac{q^{n^2}}{(zq; q)_n \cdot (q/z; q)_n}$$

At $z = 1$, the rank generating function reduces to the ordinary partition generating
function, since the rank statistic partitions all partitions of $n$ into classes
that sum to $p(n)$.

In [6]:
from q_kangaroo import rank_gf

# At z=1, rank_gf equals the partition generating function
rgf = rank_gf(s, 1, 1, 25)
rgf

1 + q + 2*q^2 + 3*q^3 + 5*q^4 + 7*q^5 + 11*q^6 + 15*q^7 + 22*q^8 + 30*q^9 + 42*q^10 + 56*q^11 + 77*q^12 + 101*q^13 + 135*q^14 + 176*q^15 + 231*q^16 + 297*q^17 + 385*q^18 + 490*q^19 + 627*q^20 + 792*q^21 + 1002*q^22 + 1255*q^23 + 1575*q^24 + O(q^25)

The output matches `partition_gf(s, 25)` exactly, confirming that at $z=1$ the rank
generating function reduces to $\sum p(n) q^n$.

## Dyson's Conjecture

In 1944, Dyson conjectured that for $0 \le t \le 4$, the number of partitions of
$5n+4$ with rank congruent to $t \pmod{5}$ equals $p(5n+4)/5$. In other words, the
rank statistic splits the partitions of $5n+4$ into 5 equal classes, providing a
**combinatorial witness** for the congruence $p(5n+4) \equiv 0 \pmod{5}$.

This was proved by Atkin and Swinnerton-Dyer in 1954. The rank similarly explains
the mod 7 congruence: partitions of $7n+5$ split into 7 equal rank classes. However,
the rank does **not** explain the mod 11 congruence -- a different statistic was needed.

## Crank Generating Function

The **crank** statistic was conjectured by Dyson (who called it the "crank") and
realized by Andrews and Garvan in 1988. For a partition $\lambda$, the crank is:

- If $\lambda$ has no ones: the crank is the largest part.
- If $\lambda$ has ones: the crank is $\mu(\lambda) - \omega(\lambda)$, where
  $\omega(\lambda)$ is the number of ones and $\mu(\lambda)$ is the number of
  parts larger than $\omega(\lambda)$.

The crank generating function is:

$$C(z, q) = \frac{(q; q)_\infty}{(zq; q)_\infty \cdot (q/z; q)_\infty}$$

At $z = 1$, like the rank, it reduces to the partition generating function:

In [7]:
from q_kangaroo import crank_gf

# At z=1, crank_gf also equals the partition generating function
cgf = crank_gf(s, 1, 1, 25)
cgf

1 + q + 2*q^2 + 3*q^3 + 5*q^4 + 7*q^5 + 11*q^6 + 15*q^7 + 22*q^8 + 30*q^9 + 42*q^10 + 56*q^11 + 77*q^12 + 101*q^13 + 135*q^14 + 176*q^15 + 231*q^16 + 297*q^17 + 385*q^18 + 490*q^19 + 627*q^20 + 792*q^21 + 1002*q^22 + 1255*q^23 + 1575*q^24 + O(q^25)

The key advantage of the crank over the rank is that the crank explains **all three**
Ramanujan congruences (mod 5, 7, and 11). The rank only works for mod 5 and mod 7.
Andrews and Garvan proved that the crank splits partitions of $5n+4$, $7n+5$, and
$11n+6$ into 5, 7, and 11 equal classes respectively.

## Prodmake: Infinite Product Analysis

Andrews' `prodmake` algorithm takes a formal power series and determines exponents
$a_n$ such that

$$f(q) = \prod_{n=1}^{\infty} (1 - q^n)^{-a_n}$$

This is the inverse of infinite product expansion: given a series, recover its
product representation.

In [8]:
from q_kangaroo import prodmake

# Analyze the partition generating function
pm = prodmake(partition_gf(s, 50), 20)
print("Prodmake factors (a_n where f = prod (1-q^n)^{-a_n}):")
print(pm["factors"])

Prodmake factors (a_n where f = prod (1-q^n)^{-a_n}):
{1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1}


Every exponent $a_n = 1$, confirming the classical formula

$$\sum_{n=0}^{\infty} p(n) q^n = \prod_{k=1}^{\infty} \frac{1}{1 - q^k} = \prod_{k=1}^{\infty} (1 - q^k)^{-1}$$

The `prodmake` algorithm recovers this structure automatically from the series
coefficients using a logarithmic derivative recurrence followed by Mobius inversion.

## Etamake: Eta-Quotient Representation

The `etamake` function groups the `prodmake` output by divisor, recovering the
Dedekind eta-quotient representation $\prod_d \eta(d\tau)^{r_d}$. Since
$\eta(\tau) = q^{1/24} \prod_{n=1}^{\infty}(1 - q^n)$, the eta function
absorbs the uniform exponent pattern.

In [9]:
from q_kangaroo import etamake

em = etamake(partition_gf(s, 50), 20)
print("Eta-quotient factors:", em["factors"])
print("q-shift:", em["q_shift"])

Eta-quotient factors: {1: -1}
q-shift: -1/24


The result $\{1: -1\}$ with $q$-shift $-1/24$ means the partition generating
function is $q^{-1/24} \cdot \eta(\tau)^{-1}$, or equivalently

$$\sum p(n) q^n = \frac{1}{(q; q)_\infty}$$

which is the classical Euler product (the $q^{-1/24}$ shift accounts for the
difference between the eta function and the bare infinite product).

## Distinct Parts and Euler's Theorem

A partition into **distinct parts** uses each part at most once (e.g., $5 = 4+1 = 3+2$).
A partition into **odd parts** uses only odd numbers (e.g., $5 = 5 = 3+1+1 = 1+1+1+1+1$).

**Euler's theorem** (1748): The number of partitions of $n$ into distinct parts equals
the number of partitions of $n$ into odd parts. This identity is reflected in the
generating functions:

$$\prod_{k=1}^{\infty}(1 + q^k) = \prod_{k=0}^{\infty} \frac{1}{1 - q^{2k+1}}$$

In [10]:
from q_kangaroo import distinct_parts_gf, odd_parts_gf

dpgf = distinct_parts_gf(s, 20)
opgf = odd_parts_gf(s, 20)
print("Distinct parts:", dpgf)
print("Odd parts:     ", opgf)
print("Equal:", repr(dpgf) == repr(opgf))

Distinct parts: 1 + q + q^2 + 2*q^3 + 2*q^4 + 3*q^5 + 4*q^6 + 5*q^7 + 6*q^8 + 8*q^9 + 10*q^10 + 12*q^11 + 15*q^12 + 18*q^13 + 22*q^14 + 27*q^15 + 32*q^16 + 38*q^17 + 46*q^18 + 54*q^19 + O(q^20)
Odd parts:      1 + q + q^2 + 2*q^3 + 2*q^4 + 3*q^5 + 4*q^6 + 5*q^7 + 6*q^8 + 8*q^9 + 10*q^10 + 12*q^11 + 15*q^12 + 18*q^13 + 22*q^14 + 27*q^15 + 32*q^16 + 38*q^17 + 46*q^18 + 54*q^19 + O(q^20)
Equal: True


The two series are identical, confirming Euler's theorem. The sequence
$1, 1, 1, 2, 2, 3, 4, 5, 6, 8, 10, 12, 15, 18, 22, 27, 32, 38, 46, 54, \ldots$
counts partitions into distinct parts (equivalently, into odd parts).

The algebraic proof follows from the identity
$(1 + q^k) = \frac{1 - q^{2k}}{1 - q^k}$, so that

$$\prod_{k=1}^{\infty}(1+q^k) = \prod_{k=1}^{\infty}\frac{1-q^{2k}}{1-q^k}
= \frac{\prod_{k=1}^{\infty}(1-q^{2k})}{\prod_{k=1}^{\infty}(1-q^k)}
= \prod_{k=0}^{\infty} \frac{1}{1 - q^{2k+1}}$$

since the even factors cancel.

## Historical Context

Ramanujan communicated these congruences to Hardy in 1919. The proofs rely on
deep properties of modular forms:

- The mod 5 and mod 7 congruences were proved by Ramanujan himself using
  properties of the Dedekind eta function.
- The mod 11 congruence required more sophisticated techniques, later
  developed by Atkin and Swinnerton-Dyer.

Dyson's **rank** of a partition, $\text{rank}(\lambda) = \text{largest part} - \text{number of parts}$,
explains the mod 5 and mod 7 congruences combinatorially. The **crank**, conjectured by
Dyson and defined by Andrews and Garvan, provides a unified combinatorial explanation
for all three congruences.

In this notebook we demonstrated q-Kangaroo's tools for partition analysis:

| Tool | Purpose |
|------|:--------|
| `partition_count` | Compute $p(n)$ via pentagonal recurrence |
| `partition_gf` | Generate $\sum p(n) q^n$ as a formal power series |
| `sift` | Extract subsequence $p(mn+j)$ |
| `findcong` | Automatic congruence discovery |
| `rank_gf` | Rank generating function $R(z, q)$ |
| `crank_gf` | Crank generating function $C(z, q)$ |
| `prodmake` | Andrews' series-to-product algorithm |
| `etamake` | Dedekind eta-quotient representation |
| `distinct_parts_gf` | $\prod(1+q^k)$ |
| `odd_parts_gf` | $\prod 1/(1-q^{2k+1})$ |